# Clustering Crypto

In [24]:
# Initial imports
import pandas as pd
import hvplot.pandas
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


### Deliverable 1: Preprocessing the Data for PCA

In [39]:
# Load the crypto_data.csv dataset.
file_path = "crypto_data.csv"
crypto_df=pd.read_csv(file_path)


In [40]:
crypto_df.tail()

,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
1247,XBC,BitcoinPlus,Scrypt,True,PoS,1.283270e+05,1000000
1248,DVTC,DivotyCoin,Scrypt,False,PoW/PoS,2.149121e+07,100000000
1249,GIOT,Giotto Coin,Scrypt,False,PoW/PoS,NaN,233100000
1250,OPSC,OpenSourceCoin,SHA-256,False,PoW/PoS,NaN,21000000
1251,PUNK,SteamPunk,PoS,False,PoS,NaN,40000000


In [41]:
crypto_df.dtypes

Unnamed: 0          object
CoinName            object
Algorithm           object
IsTrading             bool
ProofType           object
TotalCoinsMined    float64
TotalCoinSupply     object
dtype: object

In [42]:
# Keep all the cryptocurrencies that are being traded.
crypto_df=crypto_df.loc[crypto_df["IsTrading"]==True]
crypto_df

,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...,...,...
1243,SERO,Super Zero,Ethash,True,PoW,NaN,1000000000
1244,UOS,UOS,SHA-256,True,DPoI,NaN,1000000000
1245,BDX,Beldex,CryptoNight,True,PoW,9.802226e+08,1400222610
1246,ZEN,Horizen,Equihash,True,PoW,7.296538e+06,21000000


In [43]:
# Keep all the cryptocurrencies that have a working algorithm.
crypto_df["Algorithm"].isnull().sum()

0

In [44]:
# Remove the "IsTrading" column. 
crypto_df.drop(columns=["IsTrading"], inplace=True)


In [45]:
crypto_df

,Unnamed: 0,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,PoW,NaN,611000
4,808,808,SHA-256,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...,...
1243,SERO,Super Zero,Ethash,PoW,NaN,1000000000
1244,UOS,UOS,SHA-256,DPoI,NaN,1000000000
1245,BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
1246,ZEN,Horizen,Equihash,PoW,7.296538e+06,21000000


In [46]:

for column in crypto_df.columns:
    print(f"Column {column} has {crypto_df[column].isnull().sum()} null values")
    

Column Unnamed: 0 has 0 null values
Column CoinName has 0 null values
Column Algorithm has 0 null values
Column ProofType has 0 null values
Column TotalCoinsMined has 459 null values
Column TotalCoinSupply has 0 null values


In [47]:
# Remove rows that have at least 1 null value.    
crypto_df=crypto_df.dropna()

In [48]:
crypto_df

,Unnamed: 0,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
2,404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
4,808,808,SHA-256,PoW/PoS,0.000000e+00,0
5,1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
7,BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
...,...,...,...,...,...,...
1238,ZEPH,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000
1242,GAP,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
1245,BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
1246,ZEN,Horizen,Equihash,PoW,7.296538e+06,21000000


In [50]:
# Keep the rows where coins are mined.
crypto_df=crypto_df.loc[crypto_df["TotalCoinsMined"]>0]

In [54]:
crypto_df=crypto_df.rename(columns={"Unnamed: 0":"abbrv"})
crypto_df

,abbrv,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
2,404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
5,1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
7,BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
8,ETH,Ethereum,Ethash,PoW,1.076842e+08,0
...,...,...,...,...,...,...
1238,ZEPH,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000
1242,GAP,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
1245,BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
1246,ZEN,Horizen,Equihash,PoW,7.296538e+06,21000000


In [ ]:
crypto_df=crypto_df.set_index('abbrv')
crypto_df

In [58]:
# Create a new DataFrame that holds only the cryptocurrencies names.

name_df=crypto_df["CoinName"]
name_df

abbrv
42          42 Coin
404         404Coin
1337      EliteCoin
BTC         Bitcoin
ETH        Ethereum
           ...     
ZEPH         ZEPHYR
GAP         Gapcoin
BDX          Beldex
ZEN         Horizen
XBC     BitcoinPlus
Name: CoinName, Length: 532, dtype: object

In [59]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
crypto_df.drop(columns=["CoinName"], inplace=True)

In [60]:
crypto_df

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
abbrv,,,,
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0
...,...,...,...,...
ZEPH,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,CryptoNight,PoW,9.802226e+08,1400222610


In [61]:
# Use get_dummies() to create variables for text features.
X=pd.get_dummies(crypto_df)

In [62]:
X

,TotalCoinsMined,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,Algorithm_Cloverhash,...,TotalCoinSupply_91388946,TotalCoinSupply_92000000000,TotalCoinSupply_9354000,TotalCoinSupply_9507271,TotalCoinSupply_9736000,TotalCoinSupply_98000000,TotalCoinSupply_98100000000,TotalCoinSupply_990000000000,TotalCoinSupply_999481516,TotalCoinSupply_9999999
abbrv,,,,,,,,,,,,,,,,,,,,,
42,4.199995e+01,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
404,1.055185e+09,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZEPH,2.000000e+09,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
GAP,1.493105e+07,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
BDX,9.802226e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [63]:
# Standardize the data with StandardScaler().
X=StandardScaler().fit_transform(X)

In [64]:
X

array([[-0.11710817, -0.0433963 , -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.09396955, -0.0433963 , -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [ 0.52494561, -0.0433963 , -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       ...,
       [-0.09561336, -0.0433963 , -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.11694817, -0.0433963 , -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.11710536, -0.0433963 , -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ]])

### Deliverable 2: Reducing Data Dimensions Using PCA

In [70]:
# Using PCA to reduce dimension to three principal components.
pca = PCA(n_components=3)
pca_x = pca.fit_transform(X)

In [74]:
# Create a DataFrame with the three principal components.
pcs_df=pd.DataFrame(data=pca_x, columns=["PC 1", "PC 2", "PC 3"], index=crypto_df.index)

In [75]:
pcs_df

,PC 1,PC 2,PC 3
abbrv,,,
42,-0.267782,-0.161917,0.055350
404,-0.244659,-0.158875,0.062186
1337,0.169538,-0.150871,0.022793
BTC,-0.219227,-0.175956,0.001699
ETH,-0.081517,-0.145256,0.095202
...,...,...,...
ZEPH,2.685466,-0.113917,0.313948
GAP,-0.257439,-0.176452,0.046846
BDX,-0.108810,-0.181630,-0.089277


### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [76]:
# Create an elbow curve to find the best value for K.
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of K values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pcs_df)
    inertia.append(km.inertia_)

# Create the elbow curve
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")


C:\Users\m122353\Anaconda3\envs\PythonData\envs\mlenv\lib\site-packages\sklearn\cluster\_kmeans.py:1037: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.
  "KMeans is known to have a memory leak on Windows "


:Curve   [k]   (inertia)

Running K-Means with `k=4`

In [86]:
# Initialize the K-Means model.
km = KMeans(n_clusters=4, random_state=0)

# Fit the model
km.fit(pcs_df)

# Predict clusters
predictions = km.predict(pcs_df)

In [80]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
combine=[crypto_df, pcs_df]
clustered_df=pd.concat(combine,axis=1, join="inner")
clustered_df

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3
abbrv,,,,,,,
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.267782,-0.161917,0.055350
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.244659,-0.158875,0.062186
1337,X13,PoW/PoS,2.927942e+10,314159265359,0.169538,-0.150871,0.022793
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.219227,-0.175956,0.001699
ETH,Ethash,PoW,1.076842e+08,0,-0.081517,-0.145256,0.095202
...,...,...,...,...,...,...,...
ZEPH,SHA-256,DPoS,2.000000e+09,2000000000,2.685466,-0.113917,0.313948
GAP,Scrypt,PoW/PoS,1.493105e+07,250000000,-0.257439,-0.176452,0.046846
BDX,CryptoNight,PoW,9.802226e+08,1400222610,-0.108810,-0.181630,-0.089277


In [81]:
#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies.
combine=[clustered_df,name_df]
clustered_df=pd.concat(combine,axis=1, join="inner")
clustered_df

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,CoinName
abbrv,,,,,,,,
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.267782,-0.161917,0.055350,42 Coin
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.244659,-0.158875,0.062186,404Coin
1337,X13,PoW/PoS,2.927942e+10,314159265359,0.169538,-0.150871,0.022793,EliteCoin
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.219227,-0.175956,0.001699,Bitcoin
ETH,Ethash,PoW,1.076842e+08,0,-0.081517,-0.145256,0.095202,Ethereum
...,...,...,...,...,...,...,...,...
ZEPH,SHA-256,DPoS,2.000000e+09,2000000000,2.685466,-0.113917,0.313948,ZEPHYR
GAP,Scrypt,PoW/PoS,1.493105e+07,250000000,-0.257439,-0.176452,0.046846,Gapcoin
BDX,CryptoNight,PoW,9.802226e+08,1400222610,-0.108810,-0.181630,-0.089277,Beldex


In [87]:
#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
clustered_df["class"] = km.labels_
clustered_df.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,CoinName,class
abbrv,,,,,,,,,
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.267782,-0.161917,0.055350,42 Coin,0
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.244659,-0.158875,0.062186,404Coin,0
1337,X13,PoW/PoS,2.927942e+10,314159265359,0.169538,-0.150871,0.022793,EliteCoin,0
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.219227,-0.175956,0.001699,Bitcoin,0
ETH,Ethash,PoW,1.076842e+08,0,-0.081517,-0.145256,0.095202,Ethereum,0


In [88]:
# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

(532, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,CoinName,class
abbrv,,,,,,,,,
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.267782,-0.161917,0.055350,42 Coin,0
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.244659,-0.158875,0.062186,404Coin,0
1337,X13,PoW/PoS,2.927942e+10,314159265359,0.169538,-0.150871,0.022793,EliteCoin,0
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.219227,-0.175956,0.001699,Bitcoin,0
ETH,Ethash,PoW,1.076842e+08,0,-0.081517,-0.145256,0.095202,Ethereum,0
LTC,Scrypt,PoW,6.303924e+07,84000000,-0.252278,-0.237595,0.033695,Litecoin,0
DASH,X11,PoW/PoS,9.031294e+06,22000000,-0.377867,-0.059716,-0.002938,Dash,0
XMR,CryptoNight-V7,PoW,1.720114e+07,0,-0.080886,-0.150268,-0.001724,Monero,0
ETC,Ethash,PoW,1.133597e+08,210000000,-0.226176,-0.147043,0.094064,Ethereum Classic,0


### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [ ]:
# Creating a 3D-Scatter with the PCA data and the clusters
# YOUR CODE HERE


In [ ]:
# Create a table with tradable cryptocurrencies.
# YOUR CODE HERE

In [ ]:
# Print the total number of tradable cryptocurrencies.
# YOUR CODE HERE

In [ ]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
# YOUR CODE HERE

In [ ]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
# YOUR CODE HERE

# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
# YOUR CODE HERE

# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
# YOUR CODE HERE

plot_df.head(10)

In [ ]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
# YOUR CODE HERE
